In [ ]:
import transformers

In [1]:
from dataset import load_and_process_data, create_stock_dataset
from model import Decoder, combined_rotary_embedding, custom_mask, causal_mask
from bit.bitlinear import replace_with_bitnet_linear
from bit.RMSNorm import RMSNorm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
past_days = 29
days = 30
# 超参数
lr = 1e-4
steps = 1000
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
data = load_and_process_data()
stock_dataset = create_stock_dataset(data,seq_length=days)
train_loader = torch.utils.data.DataLoader(stock_dataset, batch_size=batch_size, shuffle=False)

In [4]:
# 输出loader的数据
for i in train_loader:
    print(i[0].shape)
    break

torch.Size([1650, 12])


In [17]:
args = {
    "d_model": 256,
    "n_head": 4,
    "dim_feedforward": 256,
    "num_layers": 3,
    "dropout": 0.1,
}
# torch模型 list
#@torch.compile
class MyModel(nn.Module):
    def __init__(self,d_model,n_head,dim_feedforward,num_layers,dropout):
        super().__init__()
        self.fc1 = nn.Linear(12, d_model)
        self.norm = RMSNorm(d_model)
        self.decoder = Decoder(**args)
        self.lm_head = nn.Linear(d_model,1)

    def forward(self, x, positions_ids,mask):
        x = self.fc1(x)
        x = self.norm(x)
        x = self.decoder(x,positions_ids,mask)
        x = self.lm_head(x)
        return x
model = MyModel(**args)

In [6]:
model = replace_with_bitnet_linear(model)

成功替换了所有 nn.Linear 层为 BitLinear 层。


In [18]:
causal = causal_mask(days*55)
pos = custom_mask(days*55,12,past_days,55)
positions_ids = combined_rotary_embedding(30, 55, args['d_model']*2//args['n_head'],500,20000)

In [19]:
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.MSELoss()

In [20]:
# 训练,用tqdm显示进度
model.train()

for step in tqdm(range(steps), desc='Steps', unit='step'):
    for x in train_loader:
        for i in tqdm(range((days - past_days) * 55), desc='Time steps', unit='time step', leave=False):
        
            optimizer.zero_grad()
            y = x[:,past_days * 55 + i,-1]
            input = x * pos[i]
            y_pred = model(input, positions_ids,causal)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            
        break
    break

Steps:   0%|          | 0/1000 [00:00<?, ?step/s]/opt/miniconda3/lib/python3.12/site-packages/torch/nn/modules/loss.py:538: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1650, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Steps:   0%|          | 0/1000 [01:47<?, ?step/s]


KeyboardInterrupt: 